## Fixes:
- sort in absolute value

In [17]:
import pandas as pd
import numpy as np
import numpy.linalg as la
import scipy.stats as ss
import sys
import py5
import scanpy as sc


In [2]:
%load_ext py5
%gui osx

In [3]:
selGene = -1
requestSelection = False
selector = None
indices = []
scrollList = None
exportBtn = None

In [4]:
def angleBetween(v1, v2):
    # Angle calculation A
#     amt = np.dot(v1, v2) / (la.norm(v1) * la.norm(v2))
#     if amt <= -1:
#         return -py5.PI
#     elif amt >= 1:
#         return 0
#     return np.arccos(amt)
    # Angle calculation B
    cosang = np.dot(v1, v2)
    sinang = la.norm(np.cross(v1, v2))
    return np.arctan2(sinang, cosang)

In [5]:
class Cell:
    def __init__(self, c, u1, u2):
        self.code = c
        self.umap1 = u1
        self.umap2 = u2
        self.proj = 0
        self.selected = False
        self.expression = []

    def normalize(self, min1, max1, min2, max2):
        self.umap1 = py5.remap(self.umap1, min1, max1, 0, 1)
        self.umap2 = py5.remap(self.umap2, min2, max2, 1, 0)
  
    def initExpression(self, numGenes):
        self.expression = [0.0] * numGenes

    def setExpression(self, i, level):
        self.expression[i] = level

    def setAllExpressions(self, levels):
        self.expression = levels
        
    def project(self, sel):
        if self.selected:
            dirv = np.array([sel.nx1 - sel.nx0, sel.ny1 - sel.ny0])
            celv = np.array([self.umap1 - sel.nx0, self.umap2 - sel.ny0])
            a = angleBetween(dirv, celv)        
            self.proj = np.cos(a) * la.norm(celv) / la.norm(dirv)
    
    def display(self, x0, y0, w, h):
        x = py5.remap(self.umap1, 0, 1, x0, x0 + w)
        y = py5.remap(self.umap2, 0, 1, y0, y0 + h)
    
        py5.no_stroke()
        if selGene != -1:
            f = py5.constrain(py5.remap(self.expression[selGene], minGeneExp, maxGeneExp, 0, 1), 0, 1)
            py5.color_mode(py5.HSB, 360, 100, 100)
            py5.fill((1 - f) * 170 + f * 233, 74, 93, 80)
            py5.color_mode(py5.RGB, 255, 255, 255)
        else:
            if self.selected:
                py5.fill(240, 118, 104, 80)
            else:
                py5.fill(150, 80)
        py5.ellipse(x, y, 5, 5)

In [6]:
class Gene():
    def __init__(self, n, i, r, p):
        self.name = n
        self.idx = i
        self.r = r
        self.rabs = abs(r)
        self.p = p

In [7]:
CLOSED = 0
SET_SPINE = 1
SET_WIDTH = 2
COMPLETED = 3

class Selector():
    def __init__(self):
        self.state = CLOSED
        self.spx0 = 0
        self.spy0 = 0
        self.spx1 = 0
        self.spy1 = 0
        self.wx = 0 
        self.wy = 0
        self.nx0 = 0 
        self.ny0 = 0        
        self.nx1 = 0
        self.ny1 = 0
        self.angle = 0
        self.x0 = 0
        self.y0 = 0
        self.w = 0
        self.h = 0
        self.tmat = np.array([[0.0, 0.0, 0.0],
                              [0.0, 0.0, 0.0]])

    def display(self):
        if self.state == CLOSED: 
            return
        
        if self.state == SET_SPINE:
            py5.stroke(240, 118, 104)
            py5.line(self.spx0, self.spy0, self.spx1, self.spy1)
        elif self.state == SET_WIDTH:
            py5.stroke(240, 118, 104)
            py5.line(self.spx0, self.spy0, self.spx1, self.spy1)
            py5.line(self.spx1, self.spy1, self.wx, self.wy)
            self.displayBox()
        else:
            self.displayBox()
    
    
    def apply(self, cell, sx0, sy0, sw, sh):
        sx = py5.remap(cell.umap1, 0, 1, sx0, sx0 + sw)
        sy = py5.remap(cell.umap2, 0, 1, sy0, sy0 + sh)
        
        # Transformation code A
#         s = np.array([sx, sy, 1])
#         t = np.matmul(self.tmat, s)
#         tx = t[0]
#         ty = t[1]

        # Transformation code B
        tx = self.multX(sx, sy)
        ty = self.multY(sx, sy)
        
        cell.selected = 0 <= tx and tx <= self.w and -self.h/2 <= ty and ty <= self.h/2

    def updateBox(self, x, y):
        self.wx = x
        self.wy = y
        
        spdir = np.array([self.spx1 - self.spx0, self.spy1 - self.spy0])
        bxdir = np.array([self.wx - self.spx1, self.wy - self.spy1])

        a = angleBetween(spdir, bxdir)
        d = np.sin(a) * la.norm(bxdir)
    
        self.angle = np.arctan2(spdir[1], spdir[0])
    
        self.h = 2 * d
        self.w = la.norm(spdir)
        self.x0 = self.spx0
        self.y0 = self.spy0
    
        # Transformation code A
#         s = np.sin(-self.angle)
#         c = np.cos(-self.angle)
#         tx = -self.x0 * c + self.y0 * s
#         ty = -self.x0 * s - self.y0 * c
#         self.tmat = np.array([[c, -s, tx],
#                               [s,  c, ty]])

        # Transformation code B
        self.reset()
        self.rotate(-self.angle)
        self.translate(-self.x0, -self.y0)

    def normalize(self, sx0, sy0, sw, sh):
        self.nx0 = py5.remap(self.spx0, sx0, sx0 + sw, 0, 1)
        self.nx1 = py5.remap(self.spx1, sx0, sx0 + sw, 0, 1)
        self.ny0 = py5.remap(self.spy0, sy0, sy0 + sh, 0, 1)
        self.ny1 = py5.remap(self.spy1, sy0, sy0 + sh, 0, 1)
  
    def displayBox(self):
        py5.stroke(240, 118, 104)
        py5.no_fill()
        py5.push_matrix()
        py5.translate(self.x0, self.y0)
        py5.rotate(self.angle)
        py5.rect(0, -self.h/2, self.w, self.h)
        py5.pop_matrix()
        
    def press(self, x, y):
        if self.state == CLOSED or self.state == COMPLETED:
            self.state = SET_SPINE
        elif self.state == SET_SPINE:
            self.state = SET_WIDTH

        if self.state == SET_SPINE:
            self.spx0 = self.spx1 = self.wx = x
            self.spy0 = self.spy1 = self.wy = y
            self.angle = 0
            self.w = self.h = 0
  
    def drag(self, x, y):
        if self.state == SET_SPINE:
            self.spx1 = x
            self.spy1 = y      

    def move(self, x, y):
        if self.state == SET_WIDTH:
            self.updateBox(x, y)

    def release(self, x, y):
        if self.state == SET_SPINE:
            self.spx1 = x
            self.spy1 = y
            if self.spx1 != self.spx0 or self.spy1 != self.spy0:
                self.state = SET_WIDTH
                self.wx = self.spx1
                self.wy = self.spy1
            else:
                self.state = CLOSED
        elif self.state == SET_WIDTH:
            self.updateBox(x, y)
            self.state = COMPLETED
            global requestSelection
            requestSelection = True
            
    def multX(self, x, y):
        return self.tmat[0][0] * x + self.tmat[0][1] * y + self.tmat[0][2]
            
    def multY(self, x, y):
        return self.tmat[1][0] * x + self.tmat[1][1] * y + self.tmat[1][2]
    
    def reset(self):
        self.tmat = np.array([[1.0, 0.0, 0.0],
                              [0.0, 1.0, 0.0]])        
            
    def rotate(self, angle):
        s = np.sin(angle);
        c = np.cos(angle);
        
        temp1 = self.tmat[0][0]
        temp2 = self.tmat[0][1]
        self.tmat[0][0] =  c * temp1 + s * temp2
        self.tmat[0][1] = -s * temp1 + c * temp2
        temp1 = self.tmat[1][0]
        temp2 = self.tmat[1][1]
        self.tmat[1][0] =  c * temp1 + s * temp2
        self.tmat[1][1] = -s * temp1 + c * temp2
        
    def translate(self, tx, ty):
        self.tmat[0][2] = tx*self.tmat[0][0] + ty*self.tmat[0][1] + self.tmat[0][2]
        self.tmat[1][2] = tx*self.tmat[1][0] + ty*self.tmat[1][1] + self.tmat[1][2]

In [8]:
class Button:
    def __init__(self, x, y, w, h, l):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.label = l
  
    def display(self):
        py5.no_stroke()
        py5.fill(120)
        py5.rect(self.x, self.y, self.w, self.h, 15)
    
        py5.fill(255)
        py5.text(self.label, self.x, self.y, self.w, self.h)
  
    def contains(self, mx, my):
        return self.x <= mx and mx <= self.x + self.w and self.y <= my and my <= self.y + self.h

In [9]:
itemHeight = 50
itemSpace = 10

class ScrollableList:    
    def __init__(self, x, y, w, h):
        self.x = x 
        self.y = y 
        self.w = w
        self.h = h
        self.scrollbar = None
        self.genes = []
        self.selItem = -1
        self.dragged = False
  
    def setList(self, genes):
        self.genes = genes
        self.scrollbar = ScrollBar(50 * len(genes), 0.1 * self.w, self.w, self.h)
        self.selItem = -1
  
    def display(self):
        if not self.genes or len(self.genes) == 0: return
    
        py5.push_matrix()
        py5.translate(self.x, self.y)
        py5.push_matrix()
        py5.translate(0, self.scrollbar.translateY)
        py5.no_stroke()
        for i in range(0, len(self.genes)):
            py5.fill(210)
            rx = 20
            ry = i * itemHeight + itemSpace
            rw = self.w - 40
            rh = itemHeight - itemSpace
            if self.selItem == i:
                py5.stroke(240, 118, 104)
            else:
                py5.no_stroke()
            py5.rect(rx, ry, rw, rh)
            py5.fill(120)
            gene = self.genes[i]
            text = gene.name + " " + "{:1.2f}".format(gene.r)
            py5.text(text, rx, ry, rw, rh)
        py5.pop_matrix()
        self.scrollbar.display()
        py5.pop_matrix()
  
    def press(self):
        self.dragged = False
        self.scrollbar.setOpen()

    def drag(self, my, pmy):
        self.dragged = True
        self.scrollbar.update(pmy - my)

    def release(self, my):
        self.scrollbar.setClose()
        if not self.dragged:
            l = my - self.scrollbar.translateY
            self.selItem = int(l / itemHeight)

        if self.selItem != -1:
            return self.genes[self.selItem].idx  
        else:
            return -1

class ScrollBar:    
    def __init__(self, th, bw, lw, lh):
        self.totalHeight = th
        self.barWidth = bw
        self.translateY = 0
        self.opacity = 0    
        self.listWidth = lw
        self.listHeight = lh

    def setOpen(self):
        self.opacity = 150

    def setClose(self):
        self.opacity = 0

    def update(self, dy):
        if self.totalHeight + self.translateY + dy > self.listHeight:
            self.translateY += dy
            if self.translateY > 0: self.translateY = 0

    def display(self):
        if 0 < self.opacity:
            frac = self.listHeight / self.totalHeight
            x = self.listWidth - 1.5 * self.barWidth
            y = py5.remap(self.translateY / self.totalHeight, -1, 0, self.listHeight, 0)
            w = self.barWidth
            h = frac * self.listHeight
            py5.push_style()
            py5.no_stroke()
            py5.fill(150, self.opacity)
            py5.rect(x, y, w, h, 0.2 * w)
            py5.pop_style()

In [10]:
def showUMAPScatter():
    global requestSelection
    
    x0 = 25
    y0 = 25
    w = py5.width/2 - 50
    h = py5.height - 50

    global indices
    if requestSelection:
        indices = []
        selector.normalize(x0, y0, w, h)
  
    for idx in range(0, len(cells)):
        cell = cells[idx]
        if requestSelection:            
            selector.apply(cell, x0, y0, w, h)
            cell.project(selector)
            if cell.selected:
                indices += [idx]
        cell.display(x0, y0, w, h)
  
    py5.stroke_weight(2)
    py5.stroke(120)
    py5.no_fill()
    py5.rect(x0 - 2.5, y0 - 2.5, w + 5, h + 5)
    
    if selGene != -1:
        py5.no_stroke()
        for i in range(0, 20):
            f = py5.remap(i, 0, 19, 0, 1)
            py5.color_mode(py5.HSB, 360, 100, 100)
            py5.fill((1 - f) * 170 + f * 233, 74, 93, 80)
            py5.color_mode(py5.RGB, 255, 255, 255)
            x = py5.remap(f, 0, 1, x0 + 20, x0 + 120)
            py5.rect(x, y0 + 20, 100.0/19, 30)
        py5.fill(130)
        py5.text("Max exp.", x0 + 160, y0 + 35)

    py5.fill(130)
    py5.text("UMAP1", x0, y0 + h + 2.5/2, w, py5.height - y0 - h)
    py5.push_matrix()
    py5.translate((x0-2.5)/2, y0 + h/2)
    py5.rotate(-py5.HALF_PI)
    py5.text("UMAP2", 0, 0)
    py5.pop_matrix()
    
def showGeneScatter():
    x0 = py5.width/2 + 200 + 50
    w = py5.width - x0 - 100
    h = w
    y0 = (py5.height - h) / 2
  
    for i in range(0, len(indices)):      
        idx = indices[i]
        cell = cells[idx]
        x = py5.remap(cell.proj, 0, 1, x0 + 5, x0 + w - 5)
        y = py5.remap(cell.expression[selGene], minGeneExp, maxGeneExp, y0 + w - 5, y0 + 5)
        py5.no_stroke()
        py5.fill(150, 80)
        py5.ellipse(x, y, 10, 10)

    py5.fill(100)
    py5.text(geneNames[selGene], x0, 0, w, y0)
  
    py5.stroke_weight(2)
    py5.stroke(120)
    py5.no_fill()
    py5.rect(x0, y0, w, h)
  
    py5.fill(130)
    py5.text("{:1.2f}".format(maxGeneExp), x0 - 20, y0 + 5)
    py5.text("{:1.2f}".format(minGeneExp), x0 - 20, y0 + h - 5)
    py5.push_matrix()
    py5.translate(x0 - 20, y0 + h/2)
    py5.rotate(-py5.HALF_PI)
    py5.text("Expression", 0, 0)
    py5.pop_matrix()
  
    py5.text("0", x0 + 5, y0 + h + 15)
    py5.text("1", x0 + w - 5, y0 + h + 15)
    py5.text("Projection", x0 + 5, y0 + h + 10, w - 10, 20)

In [11]:
def calculateGeneCorrelations():
    print("Selected", len(indices), "cells")

    print("Calculating correlations...") 

#     rows = []
#     for i in range(0, len(indices)):
#         c = indices[i]
#         cell = cells[c]
#         row = [cell.proj] + cell.expression
#         rows += [row]
#     data = pd.DataFrame.from_records(rows)
    
#     # https://stackoverflow.com/a/24469099
#     # Pearson correlation and P-values between all columns (first colum the projection, the rest the genes)
#     n = 1 + len(geneNames)    
#     r = data.corr().to_numpy()
#     t = r*np.sqrt((n-2)/(1-r*r))

    # Or, one pair at the time...
#     r, p = scipy.stats.pearsonr(x, y)

    global sortedGenes
    sortedGenes = []
    
    vproj = []
    rexpr = []
    for i in range(0, len(indices)):
        c = indices[i]
        cell = cells[c]
        vproj += [cell.proj]
        rexpr += [cell.expression]
    dexpr = pd.DataFrame.from_records(rexpr)    
    for g in range (0, len(geneNames)):
        r, p = ss.pearsonr(vproj, dexpr[g])
        if pearsonsThreshold <= abs(r) and p <= pvalueThreshold:
            gene = Gene(geneNames[g], g, r, p)
            sortedGenes += [gene]


#     global sortedGenes
#     sortedGenes = []
#     for g in range(0, len(geneNames)):
#         ri = r[0, 1 + g]
#         pi = t[0, 1 + g]
#         if pearsonsThreshold <= abs(ri) and pi <= pvalueThreshold:
#             gene = Gene(geneNames[g], g, ri, pi)
#             sortedGenes += [gene]
            
    sortedGenes.sort(key=lambda x: x.r, reverse=False)

    scrollList.setList(sortedGenes)
    
    global requestSelection
    requestSelection = False
    
    global selGene
    selGene = -1
    print("Done")
    
def calculateGeneMinMax():
    global minGeneExp
    global maxGeneExp
    minGeneExp = sys.float_info.max
    maxGeneExp = sys.float_info.min
    for i in range(0, len(indices)):
        idx = indices[i]
        cell = cells[idx]
        exp = cell.expression[selGene]
        minGeneExp = min(minGeneExp, exp)
        maxGeneExp = max(maxGeneExp, exp)   
    print("Min/max expression level for gene", geneNames[selGene], minGeneExp, maxGeneExp)

def exportData():
    print("EXPORTING DATA...")
    
    global selected_cells
    global significant_genes
    global selected_gene_name
    global selected_gene_cell_data
    
    rows = []
    for i in range(0, len(indices)):
        idx = indices[i]
        cell = cells[idx]
        row = [cell.code, cell.proj]
        rows += [row]
    global selected_cells
    selected_cells = pd.DataFrame.from_records(rows, columns=['index', 'proj'])

    rows = []
    for gene in sortedGenes:
        row = [gene.r, gene.p]
        rows += [row]
    significant_genes = pd.DataFrame.from_records(rows, columns=['R', 'P'])

    selected_gene_name = geneNames[selGene]
    
    rows = []
    for i in range(0, len(indices)):
        idx = indices[i]
        cell = cells[idx]
        row = [cell.code, cell.proj, cell.expression[selGene]]
        rows += [row]        
    selected_gene_cell_data = pd.DataFrame.from_records(rows, columns=['index', 'proj', 'exp'])        
    
    print("BYE")
    py5.exit_sketch()

In [12]:
def initUI():
    global selector
    global scrollList
    global exportBtn
    
    selector = Selector()
    scrollList = ScrollableList(py5.width/2, 0, 200, py5.height)  
    w = py5.width - (py5.width/2 + 200)
    exportBtn = Button(py5.width/2 + 200 + w/2 - 75, py5.height - 75, 100, 30, "EXPORT")    

In [13]:
def settings():
    py5.size(1600, 800, py5.P2D)
    
def setup():
    initUI()    
    py5.text_align(py5.CENTER, py5.CENTER)
    py5.text_font(py5.create_font("Helvetica", 14))
    
def draw():
    global selGene
    global selector
    global scrollList
    global exportBtn
    global requestSelection
    
    py5.background(255)
    showUMAPScatter()
    
    if requestSelection and 0 < len(indices):
        calculateGeneCorrelations()
    
    selector.display()
    scrollList.display()
    
    if selGene != -1:
        showGeneScatter()
    exportBtn.display()
    
def mouse_pressed():
    global selector
    global scrollList
    
    if py5.mouse_x < py5.width/2:
        selector.press(py5.mouse_x, py5.mouse_y)
    elif py5.mouse_x < py5.width/2 + 200:
        scrollList.press()

def mouse_dragged():
    global selector
    global scrollList
    
    if py5.mouse_x < py5.width/2:
        selector.drag(py5.mouse_x, py5.mouse_y)
    elif py5.mouse_x < py5.width/2 + 200:
        scrollList.drag(py5.mouse_y, py5.pmouse_y)
    
def mouse_moved():
    global selector
    
    if py5.mouse_x < py5.width/2:
        selector.move(py5.mouse_x, py5.mouse_y)

def mouse_released():
    global selGene
    global selector
    global scrollList
    global exportBtn
    
    if py5.mouse_x < py5.width/2:
        selector.release(py5.mouse_x, py5.mouse_y)
    elif py5.mouse_x < py5.width/2 + 200:
        sel = scrollList.release(py5.mouse_y)
        if sel != -1 and sel != selGene:
            selGene = sel
            print("Selected gene", selGene)
            calculateGeneMinMax()
 
    elif exportBtn.contains(py5.mouse_x, py5.mouse_y):
        exportData()

In [14]:
minGeneExp = 0
maxGeneExp = 0

pearsonsThreshold = 0.1
pvalueThreshold = 0.05


In [15]:
cells = []
geneNames = []
sortedGenes = []

selected_cells = []
significant_genes = []
selected_gene_name = ''
selected_gene_cell_data = ''

In [18]:
results_T_file = '../data/pbmc68k_Tcells.h5ad'  # the file that will store the analysis results
adata_T = sc.read(results_T_file)


In [43]:
cell_data = pd.DataFrame(adata_T.obsm['X_umap'], index=adata_T.obs.index, columns = ['UMAP_1', 'UMAP_2'])
cell_data['index'] = cell_data.index
cell_data.index = np.arange(cell_data.shape[0])
cell_data.head()

,UMAP_1,UMAP_2,index
0,9.244787,8.620283,AAACATACCGGAGA-1
1,6.445797,6.665294,AAACATACTGTCTT-1
2,2.436583,5.597902,AAACATACTTCTAC-1
3,-0.981794,3.794023,AAACATTGCTTCGC-1
4,10.567924,1.621601,AAACATTGGTTCAG-1


In [48]:
print("LOADING UMAP DATA...")

min1 = cell_data["UMAP_1"].min()
max1 = cell_data["UMAP_1"].max()
min2 = cell_data["UMAP_2"].min()
max2 = cell_data["UMAP_2"].max()

cells = []
for row in cell_data.itertuples():
    cell = Cell(row.index, row.UMAP_1, row.UMAP_2)
    cell.normalize(min1, max1, min2, max2)
    cells += [cell]

print("DONE: LOADED", len(cells), "CELLS")

LOADING UMAP DATA...
DONE: LOADED 32168 CELLS


In [46]:
expr_data = pd.DataFrame(adata_T.X.todense(), index=adata_T.obs.index, columns=adata_T.var.index)
expr_data = expr_data.transpose()
expr_data.index.name = 'index'
expr_data.head()


,AAACATACCGGAGA-1,AAACATACTGTCTT-1,AAACATACTTCTAC-1,AAACATTGCTTCGC-1,AAACATTGGTTCAG-1,AAACATTGTCTTCA-1,AAACCGTGACAGTC-1,AAACCGTGCCCAAA-1,AAACCGTGCGATAC-1,AAACCGTGTACAGC-1,...,TTTGACTGAAGTAG-8,TTTGACTGACGTTG-8,TTTGACTGACTAGC-8,TTTGACTGCCCAAA-8,TTTGACTGCTTTAC-8,TTTGACTGTCGTTT-8,TTTGACTGTGCTAG-8,TTTGCATGCTAGCA-8,TTTGCATGCTGCAA-8,TTTGCATGTGGTAC-8
index,,,,,,,,,,,,,,,,,,,,,
LINC00115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FAM41C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NOC2L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.16312,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
KLHL17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PLEKHN1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
cells[0].code

'AAACATACCGGAGA-1'

In [51]:
print("LOADING GENE EXPRESSION DATA...")

codes = expr_data.columns.tolist()
codes.reverse()
geneNames = expr_data.index.tolist()

for cell in cells:
    if cell.code != codes.pop():
        raise Exception('ERROR: data mistmatch at cell', cell.code)
    cell.setAllExpressions(expr_data[cell.code].tolist())
        
print("DONE: LOADED", len(geneNames), "GENES FOR", len(cells), "CELLS")

LOADING GENE EXPRESSION DATA...
DONE: LOADED 12244 GENES FOR 32168 CELLS


In [52]:
py5.run_sketch()

KeyboardInterrupt: 

In [32]:
selected_cells

,index,proj
0,AAGATTACCGCCTT-1,0.633023
1,AAGCCATGAACTGC-1,0.453241
2,AATGCGTGGACGGA-1,0.725658
3,AATTACGAATTCCT-1,0.673651
4,ACCCGTTGCTTCTA-1,0.105879
5,ACGAGGGACAGGAG-1,0.364916
6,ACGTCGCTCTATTC-1,0.961825
7,ACGTGATGCCATGA-1,0.584993
8,ACTTAAGATTACTC-1,0.739217
9,AGCACTGATGCTTT-1,0.520015


In [33]:
significant_genes

,R,P
0,-0.731967,2.632655e-07
1,-0.731955,2.634355e-07
2,-0.731954,2.634562e-07
3,-0.731843,2.651060e-07
4,-0.731659,2.678689e-07
...,...,...
504,0.741972,1.478045e-07
505,0.747766,1.045282e-07
506,0.778700,1.386987e-08
507,0.835364,1.271514e-10


In [25]:
selected_gene_name

'GZMB'

In [26]:
selected_gene_cell_data

,index,proj,exp
0,AAACCGTGTATGCG-1,0.346370,4.289369
1,AACCTTACGCGAGA-1,0.093997,3.685756
2,AACGTCGAGTATCG-1,0.201407,3.242313
3,AAGATTACCTCAAG-1,0.023394,3.985905
4,AAGCAAGAGCTTAG-1,0.684119,2.959278
...,...,...,...
122,TTCCAAACTCCCAC-1,0.338375,3.623071
123,TTCCCACTTGAGGG-1,0.169578,2.576919
124,TTCGTATGGATAGA-1,0.517675,2.372028
125,TTCTGATGGAGACG-1,0.083587,4.132404
